# Neuro Painter™

## Imports

In [1]:
import time
import json
import random
import numpy as np
import tensorflow as tf

from libs import utils, gif
from libs.group_norm import GroupNormalization

from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Flatten, Reshape, Add, Multiply, Activation, Lambda
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.objectives import mean_squared_error, mean_absolute_error
from keras.callbacks import ModelCheckpoint, LambdaCallback

from keras import optimizers
from keras import backend as K

/Users/mlacy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load Image

In [ ]:
DIRECTORY = 'roadtrip'
IMAGE = "1.jpg"
SIZE = 256
OUTPUT_SIZE = 256
CHANNELS = 3

FILENAME = DIRECTORY + '/' + IMAGE
FEATURES = SIZE*SIZE*CHANNELS
MODEL_NAME = DIRECTORY + '-' + os.path.splitext(IMAGE)[0] +'-'+str(SIZE)

import os
print()

In [15]:
XS, YS, IMG = utils.load_image(FILENAME, SIZE, SIZE)

print("xs: ", XS.shape)
print("ys: ", YS.shape)
print("img: ", IMG.shape)

xs:  (65536, 2)
ys:  (65536, 3)
img:  (256, 256, 3)


/Users/mlacy/anaconda3/lib/python3.6/site-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/Users/mlacy/anaconda3/lib/python3.6/site-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


## Callbacks 

In [16]:
def gifit(epoch=None):
    if (epoch % GIF_STEPS == 0):
        print('saving gif ...')
        PREDICTION = NEUROPAINTER.predict_on_batch(XS)
        RECONS.append(np.clip(PREDICTION.reshape((OUTPUT_SIZE, OUTPUT_SIZE, 3)), 0, 1)) 
        
def saveit(epoch=None):
    if (epoch == 0):
        print('saving model ...')
        with open(MODEL_NAME+'-model.json', 'w') as f:
            json.dump(AUTOENCODER.to_json(), f, ensure_ascii=False)
            
    if (epoch % MODEL_STEPS == 0):
        print('saving weights ...')
        AUTOENCODER.save_weights(MODEL_NAME+'-weights.h5')
        
        print('saving encoder ...')
        ENCODER.save(MODEL_NAME+'-encoder.hdf5')
        
        print('saving generator ...')
        GENERATOR.save(MODEL_NAME+'-generator.hdf5')

## Training 

In [17]:
N_NEURONS = 128
N_LAYERS  = 32

INITIALIZER = 'he_normal'
ACTIVATION  = 'elu'

EPOCHS = 1000
BATCH_SIZE = 1024

GIF_STEPS = 10

In [ ]:
RECONS = []

# input
X = Input(shape=(2,))

# set the current layer
current_layer = X

# build hidden layers
for i in range(N_LAYERS):
    current_layer = Dense(N_NEURONS,activation=ACTIVATION,kernel_initializer=INITIALIZER)(current_layer)
    
# output
Y_PRED = Dense(3,activation=None,kernel_initializer=INITIALIZER)(current_layer)

# define model
NEUROPAINTER = Model(inputs=[X], outputs=[Y_PRED])

# define optimizer
ADAM = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=True)

# compile model
NEUROPAINTER.compile(optimizer=ADAM,loss='mean_squared_error')

# print summary
NEUROPAINTER.summary()

# callbacks
giffer = LambdaCallback(on_epoch_end=lambda epoch, logs: gifit(epoch))
saver = LambdaCallback(on_epoch_end=lambda epoch, logs: saveit(epoch))

# fit model
NEUROPAINTER.fit(x=XS,y=YS,batch_size=BATCH_SIZE,epochs=EPOCHS,shuffle=True,callbacks=[giffer],verbose=1)

# training gif
gif.build_gif(RECONS, show_gif=False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_100 (Dense)            (None, 128)               384       
_________________________________________________________________
dense_101 (Dense)            (None, 128)               16512     
_________________________________________________________________
dense_102 (Dense)            (None, 128)               16512     
_________________________________________________________________
dense_103 (Dense)            (None, 128)               16512     
_________________________________________________________________
dense_104 (Dense)            (None, 128)               16512     
_________________________________________________________________
dense_105 (Dense)            (None, 128)               16512     
__________

65536/65536 [==============================] - 4s 65us/step - loss: 0.0052
Epoch 39/1000
65536/65536 [==============================] - 4s 66us/step - loss: 0.0053
Epoch 40/1000
65536/65536 [==============================] - 4s 62us/step - loss: 0.0047
Epoch 41/1000
65536/65536 [==============================] - ETA: 0s - loss: 0.004 - 4s 66us/step - loss: 0.0049
saving gif ...
Epoch 42/1000
65536/65536 [==============================] - 5s 72us/step - loss: 0.0056
Epoch 43/1000
65536/65536 [==============================] - 4s 65us/step - loss: 0.0056
Epoch 44/1000
65536/65536 [==============================] - 5s 74us/step - loss: 0.0049
Epoch 45/1000
65536/65536 [==============================] - 5s 69us/step - loss: 0.0044
Epoch 46/1000
65536/65536 [==============================] - 5s 72us/step - loss: 0.0046
Epoch 47/1000
65536/65536 [==============================] - 5s 83us/step - loss: 0.0051
Epoch 48/1000
65536/65536 [==============================] - 5s 79us/step - loss: 0.0

65536/65536 [==============================] - 4s 68us/step - loss: 0.0029
Epoch 129/1000
65536/65536 [==============================] - 4s 66us/step - loss: 0.0026
Epoch 130/1000
65536/65536 [==============================] - 4s 63us/step - loss: 0.0028
Epoch 131/1000
65536/65536 [==============================] - 4s 64us/step - loss: 0.0025
saving gif ...
Epoch 132/1000
65536/65536 [==============================] - 5s 70us/step - loss: 0.0030
Epoch 133/1000
65536/65536 [==============================] - 5s 69us/step - loss: 0.0029: 0s - loss: 
Epoch 134/1000
65536/65536 [==============================] - 4s 67us/step - loss: 0.0029: 3s - loss - ETA: 2s - loss:  - ETA: 0s - l
Epoch 135/1000
65536/65536 [==============================] - 4s 66us/step - loss: 0.0026
Epoch 136/1000
65536/65536 [==============================] - 4s 66us/step - loss: 0.0033: 1s  - ETA: 0s - loss: 0.
Epoch 137/1000
65536/65536 [==============================] - 4s 66us/step - loss: 0.0028
Epoch 138/1000
65